<a href="https://colab.research.google.com/github/tb-harris/neuroscience-2024/blob/main/08_Multiple_Hypotheses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
# Load in a series of p values
# p values for all correlations beween m0 and all 40k of my genes
p_values = pd.read_hdf('/content/drive/Shareddrives/Lisman Laboratory/Lisman 2024/Neuro/toy_data/p_values_m0.hdf5')


In [7]:
import pandas as pd
import numpy as np

cell_data = pd.DataFrame(
    data=np.random.random((450, 40020)),
    index=['c' + str(i) for i in range(0, 450)],
    columns=['g' + str(i) for i in range(0, 40000)] + ['m' + str(i) for i in range(20)]
)

cell_data.to_hdf('cell_data.hdf5', key='cell_data')

gene_data = cell_data.iloc[:, :-20]
morph_data = cell_data.iloc[:, -20:]

corrs = cell_data.corrwith(cell_data['m0'])

In [8]:
corrs.sort_values(ascending=False)

m0        1.000000
g31756    0.174802
g37778    0.174402
g12145    0.169569
g26039    0.166676
            ...   
g3806    -0.165912
g17507   -0.170172
g7281    -0.184650
g10239   -0.187482
g27503   -0.198551
Length: 40020, dtype: float64

In [10]:
from scipy.stats import pearsonr
p_value = pearsonr(gene_data['g31756'], morph_data['m0'])[1]

In [11]:
p_value

0.00019414162469955535

In [4]:
p_values.sort_values(ascending=True)

g19598    0.000006
g35128    0.000016
g23695    0.000079
g19670    0.000103
g19821    0.000215
            ...   
g7967     0.999880
g312      0.999884
g29345    0.999917
g32416    0.999970
g34545    0.999986
Length: 40000, dtype: float64

Naive interpretation: Under the null hypothesis, there is a 0.0006% chance of getting our correlation value or more extreme --> We can very confidently say that gene g19598 is correlated with our morphological feature.

The p-value tells us about the significance of a single correlation. When you set the p-value threshold (alpha value) at 0.05 for a single hypothesis, you are saying that you're willing to take a 5% chance that your result might not be accurate.

When you repeat these tests, the chances that you will get a correlation that looks statistically significant isn't (false positive) increases -- when we have 40,000 results it's a near-certainty: with random data, we are almost guarunteed to have some correlations with p values below 0.05, 0.01, etc.

Some approaches we can take to address this:
* Option 1 - Only run a few tests (start with biological principles and let that guide the genes/features you pick out)
* Option 2 - Use a multiple hypothesis test correction to find an adjusted p-value that reflects the high number of hypotheses being tested

### Bonferroni Correction
Divide the alpha value (p-value threshold) by the total number of hypotheses being tested.

In [14]:
0.05 / 40000

1.25e-06

We can see that 1,941 of our correlations would be considered significant with an alpha value of 0.05 (Bad! given that this is data is random)

Without our correction, we get about 5% false positives (to be expected since our alpha value is 0.05)

In [17]:
(p_values < 0.05).sum()

1949

With our corrected alpha value, we don't get any!

In [18]:
(p_values < 0.05 / 40000).sum()

0

This correction is very conservative -- if you are still getting significant values with it, you should be good!

Note: **If you're looking at correlations for every feature and every gene**, then the number of hypotheses you're testing would be...

number of genes you're computing correlations for * number of features you're computing correlations for